#IBM Model 1

Natural Language Toolkit: IBM Model 1
Copyright (C) 2001-2013 NLTK Project
Author: Chin Yee Lee <c.lee32@student.unimelb.edu.au>
        Hengfeng Li <hengfeng12345@gmail.com>
        Ruxin Hou <r.hou@student.unimelb.edu.au>
        Calvin Tanujaya Lim <c.tanujayalim@gmail.com>
Based on earlier version by:
        Will Zhang <wilzzha@gmail.com>
        Guan Gui <ggui@student.unimelb.edu.au>
URL: <https://www.nltk.org/>
For license information, see LICENSE.TXT

Lexical translation model that ignores word order.

In IBM Model 1, word order is ignored for simplicity. As long as the
word alignments are equivalent, it doesn't matter where the word occurs
in the source or target sentence. Thus, the following three alignments
are equally likely::

    Source: je mange du jambon
    Target: i eat some ham
    Alignment: (0,0) (1,1) (2,2) (3,3)

    Source: je mange du jambon
    Target: some ham eat i
    Alignment: (0,2) (1,3) (2,1) (3,1)

    Source: du jambon je mange
    Target: eat i some ham
    Alignment: (0,3) (1,2) (2,0) (3,1)

Note that an alignment is represented here as
(word_index_in_target, word_index_in_source).

The EM algorithm used in Model 1 is:

:E step: In the training data, count how many times a source language
         word is translated into a target language word, weighted by
         the prior probability of the translation.

:M step: Estimate the new probability of translation based on the
         counts from the Expectation step.

Notations
---------

:i: Position in the source sentence
     Valid values are 0 (for NULL), 1, 2, ..., length of source sentence
:j: Position in the target sentence
     Valid values are 1, 2, ..., length of target sentence
:s: A word in the source language
:t: A word in the target language

References
----------

Philipp Koehn. 2010. Statistical Machine Translation.
Cambridge University Press, New York.

Peter E Brown, Stephen A. Della Pietra, Vincent J. Della Pietra, and
Robert L. Mercer. 1993. The Mathematics of Statistical Machine
Translation: Parameter Estimation. Computational Linguistics, 19 (2),
263-311.


## Bibliotecas

In [ ]:
import warnings
from collections import defaultdict

from nltk.translate import AlignedSent, Alignment, IBMModel
from nltk.translate.ibm_model import Counts


In [ ]:
help(AlignedSent)

Help on class AlignedSent in module nltk.translate.api:

class AlignedSent(builtins.object)
 |  AlignedSent(words, mots, alignment=None)
 |  
 |  Return an aligned sentence object, which encapsulates two sentences
 |  along with an ``Alignment`` between them.
 |  
 |      >>> from nltk.translate import AlignedSent, Alignment
 |      >>> algnsent = AlignedSent(['klein', 'ist', 'das', 'Haus'],
 |      ...     ['the', 'house', 'is', 'small'], Alignment.fromstring('0-2 1-3 2-1 3-0'))
 |      >>> algnsent.words
 |      ['klein', 'ist', 'das', 'Haus']
 |      >>> algnsent.mots
 |      ['the', 'house', 'is', 'small']
 |      >>> algnsent.alignment
 |      Alignment([(0, 2), (1, 3), (2, 1), (3, 0)])
 |      >>> from nltk.corpus import comtrans
 |      >>> print(comtrans.aligned_sents()[54])
 |      <AlignedSent: 'Weshalb also sollten...' -> 'So why should EU arm...'>
 |      >>> print(comtrans.aligned_sents()[54].alignment)
 |      0-0 0-1 1-0 2-2 3-4 3-5 4-7 5-8 6-3 7-9 8-9 9-10 9-11 10-12 11

In [ ]:
bitext = []
bitext.append(AlignedSent(['klein', 'ist', 'das', 'haus'], ['the', 'house', 'is', 'small']))
bitext.append(AlignedSent(['das', 'haus', 'ist', 'ja', 'groß'], ['the', 'house', 'is', 'big']))
bitext.append(AlignedSent(['das', 'buch', 'ist', 'ja', 'klein'], ['the', 'book', 'is', 'small']))
bitext.append(AlignedSent(['das', 'haus'], ['the', 'house']))
bitext.append(AlignedSent(['das', 'buch'], ['the', 'book']))
bitext.append(AlignedSent(['ein', 'buch'], ['a', 'book']))

## IBM Model 1 - Classe

In [ ]:
class IBMModel1(IBMModel):
  def __init__(self, sentence_aligned_corpus, iterations, probability_tables=None):
    """
    Train on ``sentence_aligned_corpus`` and create a lexical
    translation model.

    Translation direction is from ``AlignedSent.mots`` to
    ``AlignedSent.words``.

    :param sentence_aligned_corpus: Sentence-aligned parallel corpus
    :type sentence_aligned_corpus: list(AlignedSent)

    :param iterations: Number of iterations to run training algorithm
    :type iterations: int

    :param probability_tables: Optional. Use this to pass in custom
      probability values. If not specified, probabilities will be
      set to a uniform distribution, or some other sensible value.
      If specified, the following entry must be present:
      ``translation_table``.
      See ``IBMModel`` for the type and purpose of this table.
    :type probability_tables: dict[str]: object
    """
    super().__init__(sentence_aligned_corpus) 
    if probability_tables is None:
      self.set_uniform_probabilities(sentence_aligned_corpus)
    else:
      # Set user-defined probabilities
      self.translation_table = probability_tables["translation_table"]
    for n in range(0, iterations):
      self.train(sentence_aligned_corpus)

    self.align_all(sentence_aligned_corpus)

  def set_uniform_probabilities(self, sentence_aligned_corpus):
    initial_prob = 1 / len(self.trg_vocab)
    if initial_prob < IBMModel.MIN_PROB:
      warnings.warn("Target language vocabulary is too large ("
      + str(len(self.trg_vocab)) + " words). Results may be less accurate.")

    for t in self.trg_vocab:
      self.translation_table[t] = defaultdict(lambda: initial_prob)

  def train(self, parallel_corpus):
    counts = Counts()
    for aligned_sentence in parallel_corpus:
      trg_sentence = aligned_sentence.words
      src_sentence = [None] + aligned_sentence.mots

      # E step (a): Compute normalization factors to weigh counts
      total_count = self.prob_all_alignments(src_sentence, trg_sentence)

      # E step (b): Collect counts
      for t in trg_sentence:
        for s in src_sentence:
          count = self.prob_alignment_point(s, t)
          normalized_count = count / total_count[t]
          counts.t_given_s[t][s] += normalized_count
          counts.any_t_given_s[s] += normalized_count

    # M step: Update probabilities with maximum likelihood estimate
    self.maximize_lexical_translation_probabilities(counts)


  def prob_all_alignments(self, src_sentence, trg_sentence):
    """
    Computes the probability of all possible word alignments,
    expressed as a marginal distribution over target words t
      
    Each entry in the return value represents the contribution to
    the total alignment probability by the target word t.

    To obtain probability(alignment | src_sentence, trg_sentence),
    simply sum the entries in the return value.

    :return: Probability of t for all s in ``src_sentence``
    :rtype: dict(str): float
    """
    alignment_prob_for_t = defaultdict(lambda: 0.0)
    for t in trg_sentence:
      for s in src_sentence:
        alignment_prob_for_t[t] += self.prob_alignment_point(s, t)
    return alignment_prob_for_t


  def prob_alignment_point(self, s, t):
    """
    Probability that word ``t`` in the target sentence is aligned to
    word ``s`` in the source sentence
    """
    return self.translation_table[t][s]


  def prob_t_a_given_s(self, alignment_info):
    """
    Probability of target sentence and an alignment given the
    source sentence
    """
    prob = 1.0

    for j, i in enumerate(alignment_info.alignment):
      if j == 0:
        continue  # skip the dummy zeroeth element
      trg_word = alignment_info.trg_sentence[j]
      src_word = alignment_info.src_sentence[i]
      prob *= self.translation_table[trg_word][src_word]

    return max(prob, IBMModel.MIN_PROB)


  def align_all(self, parallel_corpus):
    for sentence_pair in parallel_corpus:
      self.align(sentence_pair)


  def align(self, sentence_pair):
    """
    Determines the best word alignment for one sentence pair from
    the corpus that the model was trained on.

    The best alignment will be set in ``sentence_pair`` when the
    method returns. In contrast with the internal implementation of
    IBM models, the word indices in the ``Alignment`` are zero-
    indexed, not one-indexed.

    :param sentence_pair: A sentence in the source language and its
      counterpart sentence in the target language
    :type sentence_pair: AlignedSent
    """
    best_alignment = []

    for j, trg_word in enumerate(sentence_pair.words):
      # Initialize trg_word to align with the NULL token
      best_prob = max(self.translation_table[trg_word][None], IBMModel.MIN_PROB)
      best_alignment_point = None
      for i, src_word in enumerate(sentence_pair.mots):
        align_prob = self.translation_table[trg_word][src_word]
        if align_prob >= best_prob:  # prefer newer word in case of tie
          best_prob = align_prob
          best_alignment_point = i

      best_alignment.append((j, best_alignment_point))

    sentence_pair.alignment = Alignment(best_alignment)

In [ ]:
import nltk

In [ ]:
ibm1 = nltk.translate.IBMModel1(bitext, 5)

In [ ]:
print(ibm1.translation_table['buch']['book'])


print(ibm1.translation_table['das']['book'])


print(ibm1.translation_table['buch'][None])


print(ibm1.translation_table['ja'][None])

0.8890388023094323
0.061668228784061314
0.11318685275683199
0.07278526278031176


## Nosso Corpus


In [ ]:
import pandas as pd
arquivo = '/content/Dataset_IA_Fev_2022.xlsx'
#arquivo = '/content/Frases.xlsx'

df_read = pd.read_excel(arquivo,sheet_name='Frases')

In [ ]:
df_read[:2]

,Número da frase,Contexto,Tempo (vídeo longo),Sinais Surdo,Português Surdo (teclado QWERTY),Português Surdo (teclado QWERTY correção de palavras),Português Surdo (teclado SLK),Português Intérprete (tradução),Português (retradução Angelina - palavras surdo QWERTY e mensagem final),Português (retradução Fabio - palavras surdo QWERTY e mensagem final),Português (retradução Felipe - palavras surdo QWERTY e mensagem final),Português (retradução Vagner - palavras surdo QWERTY e mensagem final),Ordem da frase (Português),Tempo Verbal (Português),Tipo de Verbo (Português),Tipo de verbo (Libras),Tempo Verbal (Libras),Uso de datilologia (Libras)
0,Teste com Surdo,Banco,------,BANCO CARTÃO PROBLEMA,banco meu cartão problema,banco meu cartão problema.,------,Meu cartão do banco está com problema.,Meu cartão do banco está com problema.\n,Meu cartão do banco está com problema.,Meu cartão do banco está com problema.,Meu cartão do banco está com problema.,O-S-V,NaN,NaN,Ancorado: Problema(Verbo de Ligação - está),Nenhum (presente),Não
1,1,Banco,23:15-23:31,CARTÃO MEU SUSTO CELULAR VI – IR BANCO CONVERS...,o meu cartao eu susto nossa paguei eu nao sabe...,o meu cartão eu susto nossa paguei eu não sabe...,------,"No meu cartão vi uma compra que não conheço, g...",Me assustei com um pagamento desconhecido que ...,Eu me assustei com uma compra no meu cartão. E...,Me assustei com um pagamento em meu cartão que...,Eu cancelo em meu cartão valor que não conheço,O-S-V,NaN,NaN,Direcional: Ver / Ir / Comprar / Explicar\n\nA...,Não,Não


In [ ]:
df_Translate = df_read[["Português Surdo (teclado QWERTY correção de palavras)","Português (retradução Fabio - palavras surdo QWERTY e mensagem final)"]] 

In [ ]:
df_Translate.rename(columns = {"Português Surdo (teclado QWERTY correção de palavras)": 'Portugues_Surdo', "Português (retradução Fabio - palavras surdo QWERTY e mensagem final)": 'Portugues'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_Translate[:25]

,Portugues_Surdo,Portugues
0,banco meu cartão problema.,Meu cartão do banco está com problema.
1,o meu cartão eu susto nossa paguei eu não sabe...,Eu me assustei com uma compra no meu cartão. E...
2,o meu cartão 4 parcelado quer pagar final.,No meu cartão tem uma compra em 4 parcelas. Qu...
3,meu INSS problema salário precisa ir conversar...,Eu tive um problema em com meu salário do INSS...
4,meu banco Santander não gosto eu quer trocar b...,Meu Banco é o Santander e eu não gosto. Eu que...
5,eu conversar combinar débito espera 15 dias ch...,Eu conversei e ficou um cartão de débito. Espe...
6,meu trabalho todos os dias 30 dias certo combi...,Eu trabalhei todos os 30 dias corretamente. O ...
7,meu salário depois celular Vivo dia 10 pagamen...,"Depois de receber o meu salário, no dia 10 per..."
8,meu cartão comprar conta e-mail outro ano eu s...,Recebi por e-mail uma compra que fiz outro ano...
9,meu carteira perder rua procurar não achar pre...,"Perdi minha carteira na rua, procurei mas não ..."


In [ ]:
df_Translate = df_Translate[:21] #lembrando que o último parâmetro é excluído do Slice!

In [ ]:
df_Translate

,Portugues_Surdo,Portugues
0,banco meu cartão problema.,Meu cartão do banco está com problema.
1,o meu cartão eu susto nossa paguei eu não sabe...,Eu me assustei com uma compra no meu cartão. E...
2,o meu cartão 4 parcelado quer pagar final.,No meu cartão tem uma compra em 4 parcelas. Qu...
3,meu INSS problema salário precisa ir conversar...,Eu tive um problema em com meu salário do INSS...
4,meu banco Santander não gosto eu quer trocar b...,Meu Banco é o Santander e eu não gosto. Eu que...
5,eu conversar combinar débito espera 15 dias ch...,Eu conversei e ficou um cartão de débito. Espe...
6,meu trabalho todos os dias 30 dias certo combi...,Eu trabalhei todos os 30 dias corretamente. O ...
7,meu salário depois celular Vivo dia 10 pagamen...,"Depois de receber o meu salário, no dia 10 per..."
8,meu cartão comprar conta e-mail outro ano eu s...,Recebi por e-mail uma compra que fiz outro ano...
9,meu carteira perder rua procurar não achar pre...,"Perdi minha carteira na rua, procurei mas não ..."


In [ ]:
df_Translate.drop(18,inplace=True)

In [ ]:
df_Translate

,Portugues_Surdo,Portugues
0,banco meu cartão problema.,Meu cartão do banco está com problema.
1,o meu cartão eu susto nossa paguei eu não sabe...,Eu me assustei com uma compra no meu cartão. E...
2,o meu cartão 4 parcelado quer pagar final.,No meu cartão tem uma compra em 4 parcelas. Qu...
3,meu INSS problema salário precisa ir conversar...,Eu tive um problema em com meu salário do INSS...
4,meu banco Santander não gosto eu quer trocar b...,Meu Banco é o Santander e eu não gosto. Eu que...
5,eu conversar combinar débito espera 15 dias ch...,Eu conversei e ficou um cartão de débito. Espe...
6,meu trabalho todos os dias 30 dias certo combi...,Eu trabalhei todos os 30 dias corretamente. O ...
7,meu salário depois celular Vivo dia 10 pagamen...,"Depois de receber o meu salário, no dia 10 per..."
8,meu cartão comprar conta e-mail outro ano eu s...,Recebi por e-mail uma compra que fiz outro ano...
9,meu carteira perder rua procurar não achar pre...,"Perdi minha carteira na rua, procurei mas não ..."


In [ ]:
df_Translate.iloc[1]

Portugues_Surdo    o meu cartão eu susto nossa paguei eu não sabe...
Portugues          Eu me assustei com uma compra no meu cartão. E...
Name: 1, dtype: object

In [ ]:
df_Translate.iloc[1][0]

'o meu cartão eu susto nossa paguei eu não saber precisa cancelar dinheiro.'

In [ ]:
df_Translate.iloc[1][1]

'Eu me assustei com uma compra no meu cartão. Eu não sei sobre esta compra. É preciso cancelar. '

In [ ]:
df_Translate.iloc[3][0].split()

['meu', 'INSS', 'problema', 'salário', 'precisa', 'ir', 'conversar', 'INSS.']

In [ ]:
len(df_Translate)

20

In [ ]:
bitext_surdo = [AlignedSent(df_Translate.iloc[x][0].split(),df_Translate.iloc[x][1].split()) for x in range(len(df_Translate))]

In [ ]:
bitext_surdo

[AlignedSent(['banco', 'meu', 'cartão', 'problema.'], ['Meu', 'cartão', 'do', 'banco', 'está', 'com', 'problema.'], Alignment([])),
 AlignedSent(['o', 'meu', 'cartão', 'eu', 'susto', 'nossa', 'paguei', 'eu', 'não', 'saber', 'precisa', 'cancelar', 'dinheiro.'], ['Eu', 'me', 'assustei', 'com', 'uma', 'compra', 'no', 'meu', 'cartão.', 'Eu', 'não', 'sei', 'sobre', 'esta', 'compra.', 'É', 'preciso', 'cancelar.'], Alignment([])),
 AlignedSent(['o', 'meu', 'cartão', '4', 'parcelado', 'quer', 'pagar', 'final.'], ['No', 'meu', 'cartão', 'tem', 'uma', 'compra', 'em', '4', 'parcelas.', 'Quero', 'pagar', 'até', 'o', 'final.'], Alignment([])),
 AlignedSent(['meu', 'INSS', 'problema', 'salário', 'precisa', 'ir', 'conversar', 'INSS.'], ['Eu', 'tive', 'um', 'problema', 'em', 'com', 'meu', 'salário', 'do', 'INSS,', 'preciso', 'conversar', 'sobre', 'ele.'], Alignment([])),
 AlignedSent(['meu', 'banco', 'Santander', 'não', 'gosto', 'eu', 'quer', 'trocar', 'banco', 'de', 'Brasil', 'gostei.'], ['Meu', 'Ban

In [ ]:
#ibm_translate = IBMModel1(bitext_surdo, 10)
ibm_translate = nltk.translate.IBMModel1(bitext_surdo, 10)

In [ ]:
print(ibm_translate.translation_table["susto"]["assustei"])
print(ibm_translate.translation_table["ligar"]["ligação"])
print(ibm_translate.translation_table["ligar"]["liguei"])
print(ibm_translate.translation_table["ver"]["vi"])
print(ibm_translate.translation_table["precisa"]["preciso"])
print(ibm_translate.translation_table["pensar"]["achei"])
print(ibm_translate.translation_table["negativo"]["negativo"])
print(ibm_translate.translation_table["pedir"]["pediu"])

0.38216546148473063
0.004830917874396135
0.004830917874396135
0.07429157519723427
0.10597277996046257
0.021066776442314836
0.12679317005188773
0.052834056143166046


In [ ]:
sentence_teste = bitext_surdo[2]
print(sentence_teste.words)
print(sentence_teste.mots)
print(sentence_teste.alignment)

['o', 'meu', 'cartão', '4', 'parcelado', 'quer', 'pagar', 'final.']
['No', 'meu', 'cartão', 'tem', 'uma', 'compra', 'em', '4', 'parcelas.', 'Quero', 'pagar', 'até', 'o', 'final.']
0-11 1-1 2-2 3-13 4-9 5-0 6-3 7-13


In [ ]:
sentence_teste = bitext_surdo[3]
print(sentence_teste.words)
print(sentence_teste.mots)
print(sentence_teste.alignment)

['meu', 'INSS', 'problema', 'salário', 'precisa', 'ir', 'conversar', 'INSS.']
['Eu', 'tive', 'um', 'problema', 'em', 'com', 'meu', 'salário', 'do', 'INSS,', 'preciso', 'conversar', 'sobre', 'ele.']
0-6 1-13 2-2 3-7 4-0 5-3 6-13 7-13


##Teste do modelo dos slides Johns Hopkins

In [ ]:
bitext_slides = []
bitext_slides.append(AlignedSent(['la','maison'],['the','house']))
bitext_slides.append(AlignedSent(['la','maison','bleu'],['the','blue','house']))
bitext_slides.append(AlignedSent(['la','fleur'],['the','flower']))

In [ ]:
bitext_slides

[AlignedSent(['la', 'maison'], ['the', 'house'], Alignment([])),
 AlignedSent(['la', 'maison', 'bleu'], ['the', 'blue', 'house'], Alignment([])),
 AlignedSent(['la', 'fleur'], ['the', 'flower'], Alignment([]))]

In [ ]:
 ibm_translate_slides1 = nltk.translate.IBMModel1(bitext_slides, 5)

In [ ]:
print(ibm_translate_slides1.translation_table['la']['the'])
print(ibm_translate_slides1.translation_table['maison']['the'])      

print(ibm_translate_slides1.translation_table['maison']['house'])      
print(ibm_translate_slides1.translation_table['bleu']['blue'])
print(ibm_translate_slides1.translation_table['bleu']['flower'])

print(ibm_translate_slides1.translation_table['fleur']['flower'])

0.706341297812428
0.23999079065357592
0.6955785675428905
0.8125329883572147
0.25
0.8826714286684298


In [ ]:
 ibm_translate_slides3 = nltk.translate.IBMModel3(bitext_slides, 5)

In [ ]:
print(ibm_translate_slides3.translation_table['la']['the'])
print(ibm_translate_slides3.translation_table['maison']['the'])      

print(ibm_translate_slides3.translation_table['maison']['house'])      
print(ibm_translate_slides3.translation_table['bleu']['blue'])
print(ibm_translate_slides3.translation_table['bleu']['flower'])

print(ibm_translate_slides3.translation_table['fleur']['flower'])

In [ ]:
sentence_teste_slides = bitext_slides[1]
print(sentence_teste_slides.words)
print(sentence_teste_slides.mots)
print(sentence_teste_slides.alignment)

['la', 'maison', 'bleu']
['the', 'blue', 'house']
0-0 1-2 2-1


# IBM Model 2

In [ ]:
import nltk

In [ ]:
ibm_translate_slides2 = nltk.translate.ibm2.IBMModel2(bitext_slides, 10)

In [ ]:
print(ibm_translate_slides2.translation_table['la']['the'])
print(ibm_translate_slides2.translation_table['maison']['the'])      

print(ibm_translate_slides2.translation_table['maison']['house'])      
print(ibm_translate_slides2.translation_table['bleu']['blue'])
print(ibm_translate_slides2.translation_table['bleu']['flower'])

print(ibm_translate_slides2.translation_table['fleur']['flower'])

## Nosso Corpus

In [ ]:
bitext_surdo2 = [AlignedSent(df_Translate.iloc[x][0].split(),df_Translate.iloc[x][1].split()) for x in range(len(df_Translate))]
bitext_surdo2

[AlignedSent(['banco', 'meu', 'cartão', 'problema.'], ['Meu', 'cartão', 'do', 'banco', 'está', 'com', 'problema.'], Alignment([])),
 AlignedSent(['o', 'meu', 'cartão', 'eu', 'susto', 'nossa', 'paguei', 'eu', 'não', 'saber', 'precisa', 'cancelar', 'dinheiro.'], ['Eu', 'me', 'assustei', 'com', 'uma', 'compra', 'no', 'meu', 'cartão.', 'Eu', 'não', 'sei', 'sobre', 'esta', 'compra.', 'É', 'preciso', 'cancelar.'], Alignment([])),
 AlignedSent(['o', 'meu', 'cartão', '4', 'parcelado', 'quer', 'pagar', 'final.'], ['No', 'meu', 'cartão', 'tem', 'uma', 'compra', 'em', '4', 'parcelas.', 'Quero', 'pagar', 'até', 'o', 'final.'], Alignment([])),
 AlignedSent(['meu', 'INSS', 'problema', 'salário', 'precisa', 'ir', 'conversar', 'INSS.'], ['Eu', 'tive', 'um', 'problema', 'em', 'com', 'meu', 'salário', 'do', 'INSS,', 'preciso', 'conversar', 'sobre', 'ele.'], Alignment([])),
 AlignedSent(['meu', 'banco', 'Santander', 'não', 'gosto', 'eu', 'quer', 'trocar', 'banco', 'de', 'Brasil', 'gostei.'], ['Meu', 'Ban

In [ ]:
ibm_translate2 = nltk.translate.ibm2.IBMModel2(bitext_surdo, 100)

In [ ]:
print(ibm_translate.translation_table["susto"]["assustei"])
print(ibm_translate.translation_table["ligar"]["ligação"])
print(ibm_translate.translation_table["ligar"]["liguei"])
print(ibm_translate.translation_table["ver"]["vi"])
print(ibm_translate.translation_table["precisa"]["preciso"])
print(ibm_translate.translation_table["pensar"]["achei"])
print(ibm_translate.translation_table["negativo"]["negativo"])
print(ibm_translate.translation_table["pedir"]["pediu"])

In [ ]:
print(ibm_translate2.translation_table["susto"]["assustei"])
print(ibm_translate2.translation_table["ligar"]["ligação"])
print(ibm_translate2.translation_table["ligar"]["liguei"])
print(ibm_translate2.translation_table["ver"]["vi"])
print(ibm_translate2.translation_table["precisa"]["preciso"])
print(ibm_translate2.translation_table["pensar"]["achei"])
print(ibm_translate2.translation_table["negativo"]["negativo"])
print(ibm_translate2.translation_table["pedir"]["pediu"])

In [ ]:
sentence_teste = bitext_surdo[2]
print(sentence_teste.words)
print(sentence_teste.mots)
print(sentence_teste.alignment)

In [ ]:
sentence_teste2 = bitext_surdo2[2]
print(sentence_teste2.words)
print(sentence_teste2.mots)
print(sentence_teste2.alignment)